### """
Created on Fri Feb 19 08:46:16 2021

@author: Schroeder
"""

# Liest holc etc. daten von bereits auf platte befindlichen csv dataframes .
# Soll Plot und Auswertung von datagrabbing trennen.


In [1]:

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display

#import TickerSelector, regression
from datetime import datetime
from datetime import date
import calendar

# Imports
from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
import scipy.stats as stats


from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
import csv
   
#import StockscreenerPlotter_stats
#import StockscreenerWinners_stats
#import downloader

import sys
import os.path
from os.path import isfile, join

from os import listdir
#from os.path import isfile, join

#currentdir = os.path.join(os.path.dirname(__file__))

###### Insert the following three lines to make any import lib in he project dir setup visible to an other
###### Directory in the project setup

currentdir = os.path.abspath('')
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
#############################################################

from lib import Indikatoren

#lieferrt bei gegebenem datatfram mit datum als index das erste Datum, das letzte, und zwei 
# dazwischen


    

## SCANNER sucht in Data Universum (etwa SP500 oder Aristokraten) nach Stocks, die ein bestimmtes Signal aufweisen. Also etwa 
## weekly MACD positiv und RSI > 75% etc.

## Gebe Pfad an, unter dem das Datenfile zu finden ist.
## Bsp: C:/Users/Schroeder/OneDrive/Trading/ETFS/RES_Aristokraten/Res/


# --------------------------------------------------------------------------
## Welcher Sektor oder index soll analysieert werden ?

In [2]:
pfad = os.path.realpath(os.path.join(parentdir, 'ETFS'))

#############################################################
who = "RES_SP500"
pfad = os.path.realpath(os.path.join(pfad, who))
###############################################################
pfad = os.path.realpath(os.path.join(pfad, 'Data'))


file  = "_ticker_names.csv"

singleticker_file=os.path.realpath(os.path.join(pfad, file))



In [3]:
exportList = pd.DataFrame(columns=['ticker',"z-score","Volume","MACDsignal","RSIlevel","Name","Industry","Sector","marketCap"])

## ---------------------------------------------------------------------

In [4]:
# DAs ist der Ticker fiel, der alle Ticker enthaelt, die gelsen werden:
# Read Tickerfile names, so all tickers in the desired directory
tkr = pd.read_csv(singleticker_file,sep=";",decimal=',',parse_dates=True,index_col=0)
        #df = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
for i in tkr["ticker"]:
    mydict={}
    file=pfad+ "/"  + i + ".csv"
    df = pd.read_csv(file,sep=";",decimal=',',parse_dates=True,index_col=0)
    #print(file)
    df["dummy"]= df["Factor"]-df["EMA100"]
    df["dummy"]=df["dummy"].fillna(0)
    Indikatoren.Zscore_rolling(df,"dummy",100)
    #df["Zscore"]=stats.zscore(df["dummy"])
    #Prüfe auf z-score
    #########################################################################
    if (df["z-score"][-1] < -2.5) or (df["z-score"][-1] >2.5):
    ########################################################################
        mydict["ticker"]=i
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
        mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
        mydict["z-score"]=df["z-score"][-1]
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        
        
    if((df["MACDdif_week"][-1] > 0) and \
        ((df["MACD_week"][-1] > 0) and (df["MACDsign_week"][-1] < 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Volume"]=df["Volume"][-1]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Long"
    if((df["MACDdif_week"][-1] < 0) and \
        ((df["MACD_week"][-1] < 0) and (df["MACDsign_week"][-1] > 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Volume"]=df["Volume"][-1]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Short"        
    
    
    # if i == "HRB":
    #     fig = px.line(df, x=df.index, y=[df["MACD_week"],df["MACDsign_week"]])
    #     fig.show()  

    if len(mydict)>0:
        exportList =exportList.append(mydict,ignore_index=True)  



In [5]:
exportList = exportList.sort_values(by="z-score",ascending=False)
exportList=exportList.fillna("-")
pd.set_option("display.max_rows",None)
display(exportList)

,ticker,z-score,Volume,MACDsignal,RSIlevel,Name,Industry,Sector,marketCap
58,ECOM,3.73639,-,-,-,"372 ChannelAdvisor Corporation Name: name, ...","372 Software—Application Name: industry, dt...","372 Technology Name: sector, dtype: object",-
57,DSX-PB,3.650826,1682,Long,-,"1279 Diana Shipping Inc. Name: name, dtype:...","1279 Marine Shipping Name: industry, dtype:...","1279 Industrials Name: sector, dtype: object",-
160,SPNT,3.182997,-,-,-,"1767 SiriusPoint Ltd. Name: name, dtype: ob...","1767 Insurance—Reinsurance Name: industry, ...","1767 Financial Services Name: sector, dtype...",-
19,APRN,3.155354,23072900,Long,-,"1564 Blue Apron Holdings, Inc. Name: name, ...","1564 Internet Retail Name: industry, dtype:...","1564 Consumer Cyclical Name: sector, dtype:...",-
197,YPF,3.048442,5528200,Long,-,"1528 YPF Sociedad Anónima Name: name, dtype...","1528 Oil & Gas Integrated Name: industry, d...","1528 Energy Name: sector, dtype: object",-
173,TKC,2.908431,-,-,-,1509 Turkcell Iletisim Hizmetleri A.S. Name...,"1509 Telecom Services Name: industry, dtype...","1509 Communication Services Name: sector, d...",-
46,COE,2.883571,83400,Long,-,1344 China Online Education Group Name: nam...,1344 Education & Training Services Name: in...,"1344 Consumer Defensive Name: sector, dtype...",-
12,AKO-B,2.832317,-,-,-,"1373 Embotelladora Andina S.A. Name: name, ...",1373 Beverages—Non-Alcoholic Name: industry...,"1373 Consumer Defensive Name: sector, dtype...",-
113,MFGP,2.569105,-,-,-,1545 Micro Focus International plc Name: na...,1545 Software—Infrastructure Name: industry...,"1545 Technology Name: sector, dtype: object",-
138,PEN,2.523942,-,-,-,"923 Penumbra, Inc. Name: name, dtype: object","923 Medical Devices Name: industry, dtype: ...","923 Healthcare Name: sector, dtype: object",-
